In [8]:
import pandas as pd
from pathlib import Path # type: ignore
import plotly.io as pio
import sys
if (module_path:=str(Path(".").absolute().resolve().parent)) not in sys.path:
    sys.path.insert(0, module_path)
from sample_info import rename_mixtures, plate_dict, ont_dir, plotting_dir, mixtures2drop, getHeatmap
outdir = plotting_dir / "coverage_depth_plots/coverage_breadth_plots"
outdir.mkdir(exist_ok=True)

In [9]:
ratios = "1,10,30,100,500,1000,5000,10000".split(",")
x_levels = [f"{x}X" for x in ratios]
def generate_threshold_df():
    for plate in plate_dict.keys():
        for bed in ont_dir.joinpath(f"MixedControl-{plate}-fastqs/output/mosdepth-threshold/").glob("*.thresholds.bed.gz"):
            df = pd.read_csv(bed, sep="\t", compression='gzip')
            df["plate"] = plate
            batch = plate_dict[plate]
            df["batch"] = ": ".join(batch)
            df["mixture"] = bed.stem.split(".",1)[0]
            df = rename_mixtures(df)
            yield df

def get_threshold_df():
    df = pd.concat((x for x in generate_threshold_df()))
    df = df[~df["mixture"].isin(mixtures2drop)]
    for x_level in x_levels:
        df[f"{x_level}_ratio"] = df[x_level]/df["0X"]
    return df

In [10]:
df = get_threshold_df()
df

,#chrom,start,end,region,0X,1X,10X,30X,100X,500X,...,batch,mixture,1X_ratio,10X_ratio,30X_ratio,100X_ratio,500X_ratio,1000X_ratio,5000X_ratio,10000X_ratio
0,MN908947.3,1,265,5'UTR,264,243,241,241,241,241,...,WB: artic,0adgio1o2o3o4o5,0.920455,0.912879,0.912879,0.912879,0.912879,0.912879,0.909091,0.909091
1,MN908947.3,1,29903,whole genome,29902,29749,28931,28806,28803,28797,...,WB: artic,0adgio1o2o3o4o5,0.994883,0.967527,0.963347,0.963247,0.963046,0.954117,0.803558,0.490335
2,MN908947.3,266,13468,ORF1a,13202,13202,12787,12782,12780,12780,...,WB: artic,0adgio1o2o3o4o5,1.000000,0.968565,0.968187,0.968035,0.968035,0.957431,0.772534,0.425920
3,MN908947.3,13468,21555,ORF1b,8087,8002,7798,7691,7690,7689,...,WB: artic,0adgio1o2o3o4o5,0.989489,0.964264,0.951033,0.950909,0.950785,0.950785,0.838754,0.636454
4,MN908947.3,21563,25384,S,3821,3821,3625,3613,3613,3608,...,WB: artic,0adgio1o2o3o4o5,1.000000,0.948705,0.945564,0.945564,0.944255,0.911018,0.820989,0.379482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,MN908947.3,27394,27759,ORF7a,365,365,365,365,365,365,...,PWRB: varskip,o4-4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
10,MN908947.3,27894,28259,ORF8,365,365,365,365,365,365,...,PWRB: varskip,o4-4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
11,MN908947.3,28274,29533,N,1259,1259,1259,1259,1259,1259,...,PWRB: varskip,o4-4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.963463
12,MN908947.3,29558,29674,ORF10,116,116,116,116,116,116,...,PWRB: varskip,o4-4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.741379,0.543103


In [16]:
x_level = "100X"
ratio_col = f"{x_level}_ratio"
genes_of_interest = ["S", "whole genome"]
for gene in genes_of_interest:
    gene_df = df[df["region"] == gene]
    gene_df.to_csv(outdir/f"{gene}-{x_level}-breadth-of-coverage.csv", index=False)
    fig = getHeatmap(gene_df,field=ratio_col,labels={ratio_col:f"{x_level} ratio"},title=f"Fraction of {(gene + ' gene' if len(gene) == 1 else gene).title()} Covered at {x_level}",title_y=0.8)
    fig.show()
    pio.write_image(fig, outdir/f"{gene}-{x_level}-breadth-of-coverage.jpg", width=1400, height=700, scale=2)

In [12]:
for x_level in x_levels:
    ratio_col = f"{x_level}_ratio"
    getHeatmap(df,field=ratio_col,labels={ratio_col:f"{x_level} ratio"},title=f"Fraction of genome covered at {x_level}",title_y=0.8).show()

ValueError: Index contains duplicate entries, cannot reshape